# Create boxdata
* Input: label data
* Output: numpy array CT iamge and label (pancreas, lesion)

In [1]:
import time
import os, glob, ntpath
import logging
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import pydicom as dicom
import nrrd
import matplotlib.pyplot as plt
from datetime import datetime as ddt

In [2]:
from create_boxdata import create_boxdata, create_AD_boxdata

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
border = np.array([0, 0, 0])
box_save_path = '/home/d/pancreas/resample115_box_data/'

In [4]:
sortkey = lambda k: ntpath.basename(k).replace('.dcm', '').split('-')[-1]
sortkey_PC = lambda k: int(ntpath.basename(k).replace('.dcm', '').split('I')[-1])

# Usage
To view one simple case, just simply run `create_box_data`. It will return the numpy array of image and labels. <br>
For NP and PT, please input **sortkey**; for PC, please input **sortkey_PC**.

In [7]:
image, label = create_boxdata('/home/d/pancreas/label_data/000027/PC67/', sortkey_PC)

In [8]:
print('Image shape:', image.shape)
print('Number of content in label:', len(label))
print('Total label name:')
for item in label:
    print(' ', item[0], ':', item[1].shape)

Image shape: (110, 58, 14)
Number of content in label: 3
Total label name:
  pancreas : (110, 58, 14)
  lesion : (110, 58, 14)
  pduct : (110, 58, 14)


# Create box data and save as numpy array

In [5]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/PT*/*nrrd')):
    create_boxdata(tumorpath.replace('/label.nrrd','/'), sortkey, box_save_path=box_save_path)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=44), HTML(value='')))

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)



Done create 44 box data in 29.50455069541931 seconds


In [6]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/NP*/*nrrd')):
    create_boxdata(tumorpath.replace('/label.nrrd','/'), sortkey, box_save_path=box_save_path)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


Done create 10 box data in 4.836451530456543 seconds


In [5]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/PC*/*nrrd')):
    try:
        create_box_data(tumorpath.replace('/label.nrrd','/'), sortkey_PC, box_save_path=box_save_path)
        cnt += 1
    except:
        print(tumorpath)
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=104), HTML(value='')))

/home/d/pancreas/label_data/000282/PC134/label.nrrd
/home/d/pancreas/label_data/000322/PC252/label.nrrd
/home/d/pancreas/label_data/000263/PC78/label.nrrd
/home/d/pancreas/label_data/000136/PC38/label.nrrd
/home/d/pancreas/label_data/000149/PC2/label.nrrd
/home/d/pancreas/label_data/000309/PC167/label.nrrd
/home/d/pancreas/label_data/000309/PC168/label.nrrd
/home/d/pancreas/label_data/000309/PC166/label.nrrd
/home/d/pancreas/label_data/000350/PC306/label.nrrd
/home/d/pancreas/label_data/000339/PC273/label.nrrd
/home/d/pancreas/label_data/000329/PC349/label.nrrd
/home/d/pancreas/label_data/000140/PC3/label.nrrd
/home/d/pancreas/label_data/000312/PC246/label.nrrd
/home/d/pancreas/label_data/000351/PC301/label.nrrd
/home/d/pancreas/label_data/000285/PC116/label.nrrd
/home/d/pancreas/label_data/000147/PC55/label.nrrd
/home/d/pancreas/label_data/000334/PC319/label.nrrd
/home/d/pancreas/label_data/000144/PC34/label.nrrd
/home/d/pancreas/label_data/000144/PC32/label.nrrd
/home/d/pancreas/labe

In [5]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/no_label_data/AD*/')):
    try:
        create_AD_boxdata(tumorpath.replace('/label.nrrd','/'), sortkey, box_save_path=box_save_path)
        cnt += 1
    except:
        print(tumorpath)
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=92), HTML(value='')))

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)



Done create 92 box data in 836.549525976181 seconds


In [5]:
import nibabel as nib

In [8]:
affine = np.eye(4)
affine[0,0] = -1
affine[1,1] = -1
print(affine)

[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [9]:
for tumorpath in tqdm(glob.glob('/home/d/pancreas/no_label_data/AD*/')):
    ID = tumorpath.split('/')[-2]
    img, lbl = create_AD_boxdata(tumorpath.replace('/label.nrrd', '/'), sortkey, new_spacing=[1, 1, 1])
    image = nib.Nifti1Image(img, affine)
    image.to_filename('/home/d/pancreas/box_nifti/' + ID + 'img.nii.gz')
    label = nib.Nifti1Image(lbl, affine)
    label.to_filename('/home/d/pancreas/box_nifti/' + ID + 'lbl.nii.gz')

HBox(children=(IntProgress(value=0, max=92), HTML(value='')))

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
